In [24]:
# Step 1: Install Packages and Import Dependencies
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_metric, Dataset, DatasetDict
import numpy as np
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report

In [25]:
# Step 2: Check CUDA Availability and Device Information
# Disable CUDA
# torch.backends.cudnn.enabled = False
# torch.cuda.is_available = lambda : False

print("CUDA available:", torch.cuda.is_available())
print("Current device index:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))



CUDA available: True
Current device index: 0
Device name: NVIDIA GeForce GTX 1650 with Max-Q Design


In [26]:
# import os

# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     # os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256" 


In [27]:
# Step 3: Read and Prepare Data
def read_conll_file(file_path):
    with open(file_path, "r", encoding='utf8') as f:
        content = f.read().strip()
        sentences = content.split("\n\n")
        data = []
        for sentence in sentences:
            tokens = sentence.split("\n")
            token_data = []
            for token in tokens:
                token_data.append(token.split())
            data.append(token_data)
    return data


train_data = read_conll_file("data/hw/train.txt")
validation_data = read_conll_file("data/hw/dev.txt")
test_data = read_conll_file("data/hw/test.txt")


def convert_to_dataset(data, label_map, max_sample_len=None):
    formatted_data = {"tokens": [], "ner_tags": []}
    for sentence in data:
        tokens = [token_data[0] for token_data in sentence]
        ner_tags = [label_map[token_data[1]] for token_data in sentence]
        formatted_data["tokens"].append(tokens[:max_sample_len])
        formatted_data["ner_tags"].append(ner_tags[:max_sample_len])
    return Dataset.from_dict(formatted_data)


label_list = sorted(list(set([token_data[1] for sentence in train_data for token_data in sentence])))
label_map = {label: i for i, label in enumerate(label_list)}


train_dataset = convert_to_dataset(train_data, label_map, max_sample_len=200)
validation_dataset = convert_to_dataset(validation_data, label_map, max_sample_len=200)
test_dataset = convert_to_dataset(test_data, label_map, max_sample_len=200)


datasets = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset,
})

In [28]:
# Step 4: Initialize Tokenizer and Model (https://huggingface.co/DeepPavlov)
model_name = "DeepPavlov/distilrubert-tiny-cased-conversational-v1"#"DeepPavlov/distilrubert-tiny-cased-conversational"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/distilrubert-tiny-cased-conversational-v1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
# Step 5: Define Metrics and Tokenization Function

def compute_metrics(eval_prediction):
    predictions, labels = eval_prediction
    predictions = np.argmax(predictions, axis=2)


    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]


    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
        "classification_report": classification_report(true_labels, true_predictions),
    }


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, padding=True
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [30]:
# Step 6: Tokenize Datasets and Set Training Arguments
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    num_train_epochs=1,
    per_device_train_batch_size=4, #8,
    per_device_eval_batch_size=4, #8,
    logging_steps=100,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

Map:   0%|          | 0/7746 [00:00<?, ? examples/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map: 100%|██████████| 2582/2582 [00:00<00:00, 5929.78 examples/s]


In [31]:
# Step 7: Define Data Collator and Initialize Trainer
def data_collator(data):
    input_ids = [torch.tensor(item["input_ids"]) for item in data]
    attention_mask = [torch.tensor(item["attention_mask"]) for item in data]
    labels = [torch.tensor(item["labels"]) for item in data]


    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)


    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [32]:
# Step 8: Train the Model

trainer.train()

  6%|▌         | 107/1937 [00:02<00:51, 35.82it/s]

{'loss': 0.3798, 'learning_rate': 4.7418688693856486e-05, 'epoch': 0.05}


 11%|█         | 205/1937 [00:05<00:52, 33.30it/s]

{'loss': 0.2192, 'learning_rate': 4.4837377387712956e-05, 'epoch': 0.1}


 16%|█▌        | 307/1937 [00:08<00:44, 36.83it/s]

...

 98%|█████████▊| 1906/1937 [01:13<00:00, 34.40it/s]

{'loss': 0.0818, 'learning_rate': 9.550851832731028e-07, 'epoch': 0.98}


100%|██████████| 1937/1937 [01:14<00:00, 25.98it/s]


{'train_runtime': 74.5427, 'train_samples_per_second': 103.914, 'train_steps_per_second': 25.985, 'train_loss': 0.11499161626730128, 'epoch': 1.0}


TrainOutput(global_step=1937, training_loss=0.11499161626730128, metrics={'train_runtime': 74.5427, 'train_samples_per_second': 103.914, 'train_steps_per_second': 25.985, 'train_loss': 0.11499161626730128, 'epoch': 1.0})